<a href="https://colab.research.google.com/github/piaizv/Colaboratory/blob/main/aws_rekognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install boto3

In [ ]:
!pip install python-dotenv

In [17]:
import boto3, cv2, numpy as np, os
from dotenv import load_dotenv
from PIL import Image

In [8]:
%%writefile .env
[default]
aws_access_key_id=ASI...2X4
aws_secret_access_key=4O7...Wy1
aws_session_token=Fwo...A==
aws_bucket_source=aws...pia
aws_region=us-east-1

Writing .env


In [9]:
load_dotenv()

aws_access_key_id = os.environ.get('aws_access_key_id')
aws_secret_access_key = os.environ.get('aws_secret_access_key')
aws_session_token = os.environ.get('aws_session_token')
bucket = os.environ.get('aws_bucket_source')
region = os.environ.get('aws_region')

In [11]:
session = boto3.Session(
        aws_access_key_id = aws_access_key_id,
        aws_secret_access_key = aws_secret_access_key,
        aws_session_token = aws_session_token,
        region_name = region)
s3 = session.resource('s3')
rekognition = session.client('rekognition')

In [12]:
file = 'familia.jpg'
key = 'familia.jpg'
try:
  s3.Object(bucket, key).upload_file(file)
except Exception as e:
  print(e)

In [13]:
try:
  response = rekognition.detect_faces(Image={'S3Object': {'Bucket': bucket, 'Name': key}}, Attributes=['ALL'])
except Exception as e:
  print(e)

In [ ]:
response

In [ ]:
img = Image.open(file)
img_cv2 = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2RGBA)

height, width, _ = img_cv2.shape

for faceDetail in response['FaceDetails']:
  box = faceDetail['BoundingBox']
  x = int(width * box['Left'])
  y = int(height * box['Top'])
  w = int(width * box['Width'])
  h = int(height * box['Height'])
  roi = img_cv2[y:y+h, x:x+w]
  roi = cv2.GaussianBlur(roi, (83, 83), 30)
  img_cv2[y:y+roi.shape[0], x:x+roi.shape[1]] = roi
cv2.imwrite('familia_blur.jpg', img_cv2)